# Tidy Data In Python

Un esercizio Python motivato da questo bell'articolo di Hadley Wickham: [Tidy Data](http://vita.had.co.nz/papers/tidy-data.pdf).
Leggere questo articolo è un prerequisito per questo notebook.

Il codice in questo notebook è stato sviluppato originariamente e commentato da [Jean-François Puget](https://www.ibm.com/developerworks/community/blogs/jfp?lang=en) in un blog post su "[Tidy Data In Python](https://www.ibm.com/developerworks/community/blogs/jfp/entry/Tidy_Data_In_Python?lang=en)".

Iniziamo. 

Abbiamo bisogno soltanto di due package di Python.

In [30]:
import pandas as pd
import numpy as np

Mostriamo che vesione di pandas stiamo usando.

In [31]:
pd.__version__

'0.19.2'

## Introduzione

Un data set disordinato. *Disordinato* è utilizzato come nella pubblicazione di Hadley Wickham: qualsiasi data set che non è ordinato. Data set disordinati so spesso com

A messy data set.  *Messy* is used as defined in Hadley Wickham's paper: any data set that is not tidy. Data set disordinati sono spesso utili per mostrare agli umani come siano compatti. Questa forma è spesso usata nelle pubblicazioni.

In [32]:
messy = pd.DataFrame({'Nome' : ['Marco', 'Guido', 'Mela'], 
                      'Cognome' : ['Stretto', 'La Vespa', 'Mangio'], 
                      'Trattamento A' : [np.nan, 16, 3], 
                      'Trattamento B' : [2, 11, 1]})
messy

Cognome   Nome  Trattamento A  Trattamento B
0   Stretto  Marco            NaN              2
1  La Vespa  Guido           16.0             11
2    Mangio   Mela            3.0              1

Spesso le persone preferiscono fare la trasposizione di un data set di questo tipo.

In [33]:
messy.T

0         1       2
Cognome        Stretto  La Vespa  Mangio
Nome             Marco     Guido    Mela
Trattamento A      NaN        16       3
Trattamento B        2        11       1

Data set disordinati non sono facilmente elaborabili da applicazioni statistiche o di *machine learning*. Questi spesso presumono che i dati siano forniti come righe di una matrice le cui colonne sono caratteristiche del campione da analizzare. Questo è ciò che è un dato ordinato. Applicando la funzione di `melt()` ai nostri dati originari, ne creiamo una versione ordinata. Mettiamo la colonna `Nome` per prima, per rendere il dato più leggibile.

In [34]:
tidy = pd.melt(messy, id_vars=['Nome','Cognome'])
tidy

Nome   Cognome       variable  value
0  Marco   Stretto  Trattamento A    NaN
1  Guido  La Vespa  Trattamento A   16.0
2   Mela    Mangio  Trattamento A    3.0
3  Marco   Stretto  Trattamento B    2.0
4  Guido  La Vespa  Trattamento B   11.0
5   Mela    Mangio  Trattamento B    1.0

I valori sono corretti, ma i nomi delle colonne non sono significativi. Per fortuna la funzione `melt()` consente anche di rinominare le colonne di output.

In [35]:
tidy = pd.melt(messy, id_vars=['Nome','Cognome'], var_name='trattamento', value_name='risultato')
tidy

Nome   Cognome    trattamento  risultato
0  Marco   Stretto  Trattamento A        NaN
1  Guido  La Vespa  Trattamento A       16.0
2   Mela    Mangio  Trattamento A        3.0
3  Marco   Stretto  Trattamento B        2.0
4  Guido  La Vespa  Trattamento B       11.0
5   Mela    Mangio  Trattamento B        1.0

## Un semplice esempio di melt

In [36]:
messy = pd.DataFrame({'riga' : ['A', 'B', 'C'], 
                      'a' : [1, 2, 3],
                      'b' : [4, 5, 6],
                      'c' : [7, 8, 9]})
messy

a  b  c riga
0  1  4  7    A
1  2  5  8    B
2  3  6  9    C

In [37]:
pd.melt(messy, id_vars='riga')

riga variable  value
0    A        a      1
1    B        a      2
2    C        a      3
3    A        b      4
4    B        b      5
5    C        b      6
6    A        c      7
7    B        c      8
8    C        c      9

In [38]:
tidy = pd.melt(messy, id_vars='riga', var_name='dimensione', value_name='lunghezza')
tidy

riga dimensione  lunghezza
0    A          a          1
1    B          a          2
2    C          a          3
3    A          b          4
4    B          b          5
5    C          b          6
6    A          c          7
7    B          c          8
8    C          c          9

Pivot è l'inverso di melt

In [39]:
messy1 = tidy.pivot(index='riga',columns='dimensione',values='lunghezza')
messy1

dimensione  a  b  c
riga               
A           1  4  7
B           2  5  8
C           3  6  9

Questo è quasi uguale al dataframe originale, con l'eccezione che la riga è usata come indice. Adesso la rimuoviamo in modo semplice.

In [40]:
messy1.reset_index(inplace=True)
messy1

dimensione riga  a  b  c
0             A  1  4  7
1             B  2  5  8
2             C  3  6  9

L'ultimo passo è quello di rimuovere il nome dal set di colonne.

In [41]:
messy1.columns.name = ''
messy1

riga  a  b  c
0    A  1  4  7
1    B  2  5  8
2    C  3  6  9

Questo dataset adesso è uguale a quello di partenza.

## Intestazioni di colonna che sono valori, non nomi di variabili

This is the first issue with messy data in Hadley's paper.  Let's first create the dataframe used as an example.  For practical reasons, it was simpler to first construct the transpose of it, then process it to get the data set used in the article.

Questo è il primo esempio di dati disordinati della pubblicazione di Hadley. Creiamo prima un dataframe da usare come esempio. Per ragioni pratiche, è più semplice costruire la versione già transposta, e poi elaborarla per renderla come quella dell'articolo.

In [42]:
messy = pd.DataFrame({'Agnostici' : [27, 34, 60, 81, 76, 137],
                      'Atei' : [12, 27, 37, 52, 35, 70],
                      'Buddisti' : [27, 21, 30, 34, 33, 58],
                      'Cattolici' : [418, 617, 732, 670, 638, 1116],
                      "Non sanno/non vogliono" : [15, 14, 15, 11, 10, 35],
                      'Evangelici protetestanti' : [575, 869, 1064, 982, 881, 1486],
                      'Induisti' : [1, 9, 7, 9, 11, 34],
                      'Historically Black Prot' : [228, 244, 236, 238, 197, 223],
                      "Testimoni di Geova" : [20, 27, 24, 24, 21, 30],
                      'Ebraici' : [19, 19, 25, 25, 30, 95],
                     })
    
def transpose(df, columns):
    df = df.T.copy()
    df.reset_index(inplace=True)
    df.columns = columns
    return df

messy = transpose(messy, ['religione', '<$10k', '$10-20k', '$20-30k', '$30-40k', '$40-50k', '$50-75k'])

messy

religione  <$10k  $10-20k  $20-30k  $30-40k  $40-50k  \
0                 Agnostici     27       34       60       81       76   
1                      Atei     12       27       37       52       35   
2                  Buddisti     27       21       30       34       33   
3                 Cattolici    418      617      732      670      638   
4                   Ebraici     19       19       25       25       30   
5  Evangelici protetestanti    575      869     1064      982      881   
6   Historically Black Prot    228      244      236      238      197   
7                  Induisti      1        9        7        9       11   
8    Non sanno/non vogliono     15       14       15       11       10   
9        Testimoni di Geova     20       27       24       24       21   

   $50-75k  
0      137  
1       70  
2       58  
3     1116  
4       95  
5     1486  
6      223  
7       34  
8       35  
9       30

La funzione `melt()` è di nuovo nostra amica. Ordiniamo il risultato per religione, per rendere il tutto più leggibile.

In [43]:
tidy = pd.melt(messy, id_vars = ['religione'], var_name='reddito', value_name='frequenza')
tidy.sort_values(by=['religione'], inplace=True)
tidy.head()

religione  reddito  frequenza
0   Agnostici    <$10k         27
30  Agnostici  $30-40k         81
40  Agnostici  $40-50k         76
50  Agnostici  $50-75k        137
10  Agnostici  $10-20k         34

## Variabili archiviate sia in righe, che in colonne


Questo esempio è un po 'più complicato. Iniziamo con importare i dati e trasformarli in un dataframe. Questi dati sono disponibili a questo URL https://github.com/hadley/tidy-data/blob/master/data/tb.csv.

L'ho clonato e reso disponibile nella cartella `data`.

Leggerlo è semplice. Rimuoviamo il prefisso `new_sp_` che appare nella maggior parte delle colonne, e rinominiamone un paio.

In [44]:
url = "https://raw.githubusercontent.com/hadley/tidy-data/master/data/tb.csv"
tb = pd.read_csv(url)
tb.columns = tb.columns.str.replace('new_sp_','')
tb.rename(columns = {'new_sp' : 'total', 'iso2' : 'country'}, 
          inplace=True)
tb.head()

country  year  total  m04  m514  m014  m1524  m2534  m3544  m4554 ...  f04  \
0      AD  1989    NaN  NaN   NaN   NaN    NaN    NaN    NaN    NaN ...  NaN   
1      AD  1990    NaN  NaN   NaN   NaN    NaN    NaN    NaN    NaN ...  NaN   
2      AD  1991    NaN  NaN   NaN   NaN    NaN    NaN    NaN    NaN ...  NaN   
3      AD  1992    NaN  NaN   NaN   NaN    NaN    NaN    NaN    NaN ...  NaN   
4      AD  1993   15.0  NaN   NaN   NaN    NaN    NaN    NaN    NaN ...  NaN   

   f514  f014  f1524  f2534  f3544  f4554  f5564  f65  fu  
0   NaN   NaN    NaN    NaN    NaN    NaN    NaN  NaN NaN  
1   NaN   NaN    NaN    NaN    NaN    NaN    NaN  NaN NaN  
2   NaN   NaN    NaN    NaN    NaN    NaN    NaN  NaN NaN  
3   NaN   NaN    NaN    NaN    NaN    NaN    NaN  NaN NaN  
4   NaN   NaN    NaN    NaN    NaN    NaN    NaN  NaN NaN  

[5 rows x 23 columns]

Usiamo l'anno 2000, e cancellaimo qualche colonna, in modo da allinearci allo schema dell'articolo di Wickham.

In [45]:
messy = tb[tb['year'] == 2000].copy()
messy.drop(['total','m04','m514','f04','f514'], axis=1, inplace=True)
messy.head(10)

country  year   m014  m1524   m2534  m3544  m4554  m5564    m65  mu  \
10       AD  2000    0.0    0.0     1.0    0.0    0.0    0.0    0.0 NaN   
36       AE  2000    2.0    4.0     4.0    6.0    5.0   12.0   10.0 NaN   
60       AF  2000   52.0  228.0   183.0  149.0  129.0   94.0   80.0 NaN   
87       AG  2000    0.0    0.0     0.0    0.0    0.0    0.0    1.0 NaN   
136      AL  2000    2.0   19.0    21.0   14.0   24.0   19.0   16.0 NaN   
165      AM  2000    2.0  152.0   130.0  131.0   63.0   26.0   21.0 NaN   
178      AN  2000    0.0    0.0     1.0    2.0    0.0    0.0    0.0 NaN   
207      AO  2000  186.0  999.0  1003.0  912.0  482.0  312.0  194.0 NaN   
236      AR  2000   97.0  278.0   594.0  402.0  419.0  368.0  330.0 NaN   
265      AS  2000    NaN    NaN     NaN    NaN    1.0    1.0    NaN NaN   

      f014   f1524   f2534  f3544  f4554  f5564    f65  fu  
10     NaN     NaN     NaN    NaN    NaN    NaN    NaN NaN  
36     3.0    16.0     1.0    3.0    0.0    0.0    4.0 NaN  
60    93.0   414.0   565.0  339.0  205.0   99.0   36.0 NaN  
87     1.0     1.0     1.0    0.0    0.0    0.0    0.0 NaN  
136    3.0    11.0    10.0    8.0    8.0    5.0   11.0 NaN  
165    1.0    24.0    27.0   24.0    8.0    8.0    4.0 NaN  
178    0.0     0.0     1.0    0.0    0.0    1.0    0.0 NaN  
207  247.0  1142.0  1091.0  844.0  417.0  200.0  120.0 NaN  
236  121.0   544.0   479.0  262.0  230.0  179.0  216.0 NaN  
265    NaN     NaN     NaN    NaN    1.0    NaN    NaN NaN

In [46]:
messy.iloc[:,:11].head(10)

country  year   m014  m1524   m2534  m3544  m4554  m5564    m65  mu   f014
10       AD  2000    0.0    0.0     1.0    0.0    0.0    0.0    0.0 NaN    NaN
36       AE  2000    2.0    4.0     4.0    6.0    5.0   12.0   10.0 NaN    3.0
60       AF  2000   52.0  228.0   183.0  149.0  129.0   94.0   80.0 NaN   93.0
87       AG  2000    0.0    0.0     0.0    0.0    0.0    0.0    1.0 NaN    1.0
136      AL  2000    2.0   19.0    21.0   14.0   24.0   19.0   16.0 NaN    3.0
165      AM  2000    2.0  152.0   130.0  131.0   63.0   26.0   21.0 NaN    1.0
178      AN  2000    0.0    0.0     1.0    2.0    0.0    0.0    0.0 NaN    0.0
207      AO  2000  186.0  999.0  1003.0  912.0  482.0  312.0  194.0 NaN  247.0
236      AR  2000   97.0  278.0   594.0  402.0  419.0  368.0  330.0 NaN  121.0
265      AS  2000    NaN    NaN     NaN    NaN    1.0    1.0    NaN NaN    NaN

La funzione `melt()` è comodo, ma non è sufficiente. Usiamola ancora.

In [47]:
molten = pd.melt(messy, id_vars=['country', 'year'], value_name='cases')
molten.sort_values(by=['year', 'country'], inplace=True)
molten.head(10)

country  year variable  cases
0         AD  2000     m014    0.0
201       AD  2000    m1524    0.0
402       AD  2000    m2534    1.0
603       AD  2000    m3544    0.0
804       AD  2000    m4554    0.0
1005      AD  2000    m5564    0.0
1206      AD  2000      m65    0.0
1407      AD  2000       mu    NaN
1608      AD  2000     f014    NaN
1809      AD  2000    f1524    NaN

Non è molto bello che le informazioni sul sesso e sull'età siano insieme in un'unica stringa nella colonna `variable`. Elaboraimo il data set per creare due colonne aggiuntive, una per il sesso e una per l'intervallo di età. Poi rimuoviamo la colonna `variable`. La forma ordinata rende semplice la rimozione dei valori in cui l'età è `u`.

In [48]:
def parse_age(s):
    s = s[1:]
    if s == '65':
        return '65+'
    else:
        return s[:-2]+'-'+s[-2:]

tidy = molten[molten['variable'] != 'mu'].copy()
tidy['sex'] = tidy['variable'].apply(lambda s: s[:1])
tidy['age'] = tidy['variable'].apply(parse_age)
tidy = tidy[['country', 'year', 'sex', 'age', 'cases']]
tidy.head(10)

country  year sex    age  cases
0         AD  2000   m   0-14    0.0
201       AD  2000   m  15-24    0.0
402       AD  2000   m  25-34    1.0
603       AD  2000   m  35-44    0.0
804       AD  2000   m  45-54    0.0
1005      AD  2000   m  55-64    0.0
1206      AD  2000   m    65+    0.0
1608      AD  2000   f   0-14    NaN
1809      AD  2000   f  15-24    NaN
2010      AD  2000   f  25-34    NaN

## Variabili archiviate sia in righe, che in colonne

Questo esempio è realmente complicato. Prima creriamo il dataframe. Questa volta, ho l'ho creato usando un array e non un dizionario, solo per il piacere di fare una cosa diversa.

In [49]:
columns = ['id', 'anno', 'mese', 'elemento', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8']
data = [['MX17004', 2010, 1, 'tmax', np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, ],
        ['MX17004', 2010, 1, 'tmin', np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, ],
        ['MX17004', 2010, 2, 'tmax', np.nan, 27.3, 24.1, np.nan, np.nan, np.nan, np.nan, np.nan, ],
        ['MX17004', 2010, 2, 'tmin', np.nan, 14.4, 14.4, np.nan, np.nan, np.nan, np.nan, np.nan, ],
        ['MX17004', 2010, 3, 'tmax', np.nan, np.nan, np.nan, np.nan, 32.1, np.nan, np.nan, np.nan, ],
        ['MX17004', 2010, 3, 'tmin', np.nan, np.nan, np.nan, np.nan, 14.2, np.nan, np.nan, np.nan, ],
        ['MX17004', 2010, 4, 'tmax', np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, ],
        ['MX17004', 2010, 4, 'tmin', np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, ],
        ['MX17004', 2010, 5, 'tmax', np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, ],
        ['MX17004', 2010, 5, 'tmin', np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan,]
       ]
messy = pd.DataFrame(data=data, columns=columns); messy

id  anno  mese elemento  d1    d2    d3  d4    d5  d6  d7  d8
0  MX17004  2010     1     tmax NaN   NaN   NaN NaN   NaN NaN NaN NaN
1  MX17004  2010     1     tmin NaN   NaN   NaN NaN   NaN NaN NaN NaN
2  MX17004  2010     2     tmax NaN  27.3  24.1 NaN   NaN NaN NaN NaN
3  MX17004  2010     2     tmin NaN  14.4  14.4 NaN   NaN NaN NaN NaN
4  MX17004  2010     3     tmax NaN   NaN   NaN NaN  32.1 NaN NaN NaN
5  MX17004  2010     3     tmin NaN   NaN   NaN NaN  14.2 NaN NaN NaN
6  MX17004  2010     4     tmax NaN   NaN   NaN NaN   NaN NaN NaN NaN
7  MX17004  2010     4     tmin NaN   NaN   NaN NaN   NaN NaN NaN NaN
8  MX17004  2010     5     tmax NaN   NaN   NaN NaN   NaN NaN NaN NaN
9  MX17004  2010     5     tmin NaN   NaN   NaN NaN   NaN NaN NaN NaN

Molti dei valori sono irrilevanti. In ogni caso non è possibile filtrare i valori NaN. Prima è necessario fare il melt del dataframe.

In [50]:
molten = pd.melt(messy, 
                 id_vars=['id', 'anno','mese','elemento',],
                 var_name='giorno');
molten.dropna(inplace=True)
molten = molten.reset_index(drop=True)
molten

id  anno  mese elemento giorno  value
0  MX17004  2010     2     tmax     d2   27.3
1  MX17004  2010     2     tmin     d2   14.4
2  MX17004  2010     2     tmax     d3   24.1
3  MX17004  2010     2     tmin     d3   14.4
4  MX17004  2010     3     tmax     d5   32.1
5  MX17004  2010     3     tmin     d5   14.2

Questo dataframe non è ancora ordinato. Per prima cosa la colonna `elemento` contiene nomi di variabili. Inoltre le colonne `year, month, day` rappresentano una sola variabile: la data. Iniziamo dal risolvere il primo problema.

In [51]:
def f(row):    
    return "%d-%02d-%02d" % (row['anno'], row['mese'], int(row['giorno'][1:]))
    
molten['data'] = molten.apply(f,axis=1)
molten = molten[['id', 'elemento','value','data']]
molten

id elemento  value        data
0  MX17004     tmax   27.3  2010-02-02
1  MX17004     tmin   14.4  2010-02-02
2  MX17004     tmax   24.1  2010-02-03
3  MX17004     tmin   14.4  2010-02-03
4  MX17004     tmax   32.1  2010-03-05
5  MX17004     tmin   14.2  2010-03-05

Adesso dobbiamo usare la funzione pivot sulla colonna `elemento`.

In [52]:
tidy = molten.pivot(index='data',columns='elemento',values='value')
tidy

elemento    tmax  tmin
data                  
2010-02-02  27.3  14.4
2010-02-03  24.1  14.4
2010-03-05  32.1  14.2

Aspetta un attimo.

Dove è finita la colonna id?

Un modo per preservarla è di farla diventare un indice con la funzione `groupby()`, e applicare la `pivot()` a ogni gruppo.

In [53]:
tidy = molten.groupby('id').apply(pd.DataFrame.pivot,
                                  index='data',
                                  columns='elemento',
                                  values='value')
tidy

elemento            tmax  tmin
id      data                  
MX17004 2010-02-02  27.3  14.4
        2010-02-03  24.1  14.4
        2010-03-05  32.1  14.2

We are almost there.  We simply have to move id back as a column with the `reset_index()`.

In [25]:
tidy.reset_index(inplace=True)
tidy

element       id        date  tmax  tmin
0        MX17004  2010-02-02  27.3  14.4
1        MX17004  2010-02-03  24.1  14.4
2        MX17004  2010-03-05  32.1  14.2

We get rid of the `element` name.

In [26]:
tidy.columns.name = ''
tidy

id        date  tmax  tmin
0  MX17004  2010-02-02  27.3  14.4
1  MX17004  2010-02-03  24.1  14.4
2  MX17004  2010-03-05  32.1  14.2

Et Voilà!

## Multiple types in one table

This example is used to illustrate two of the above problems.  

Let's create it. It is an excerpt from the Billboard top hits for 2000.

In [27]:
columns = ['year','artist','track','time','date entered','wk1','wk2','wk3',]

data = [[2000,"2,Pac","Baby Don't Cry","4:22","2000-02-26",87,82,72,],
        [2000,"2Ge+her","The Hardest Part Of ...","3:15","2000-09-02",91,87,92,],
        [2000,"3 Doors Down","Kryptonite","3:53","2000-04-08",81,70,68,],
        [2000,"98^0","Give Me Just One Nig...","3:24","2000-08-19",51,39,34,],
        [2000,"A*Teens","Dancing Queen","3:44","2000-07-08",97,97,96,],
        [2000,"Aaliyah","I Don't Wanna","4:15","2000-01-29",84,62,51,],
        [2000,"Aaliyah","Try Again","4:03","2000-03-18",59,53,38,],
        [2000,"Adams,Yolanda","Open My Heart","5:30","2000-08-26",76,76,74]
        ]

messy = pd.DataFrame(data=data, columns=columns)
messy

year         artist                    track  time date entered  wk1  wk2  \
0  2000          2,Pac           Baby Don't Cry  4:22   2000-02-26   87   82   
1  2000        2Ge+her  The Hardest Part Of ...  3:15   2000-09-02   91   87   
2  2000   3 Doors Down               Kryptonite  3:53   2000-04-08   81   70   
3  2000           98^0  Give Me Just One Nig...  3:24   2000-08-19   51   39   
4  2000        A*Teens            Dancing Queen  3:44   2000-07-08   97   97   
5  2000        Aaliyah            I Don't Wanna  4:15   2000-01-29   84   62   
6  2000        Aaliyah                Try Again  4:03   2000-03-18   59   53   
7  2000  Adams,Yolanda            Open My Heart  5:30   2000-08-26   76   76   

   wk3  
0   72  
1   92  
2   68  
3   34  
4   96  
5   51  
6   38  
7   74

This dataset is messy because there are several observations per row, in the columns wk1, wk2, wk3.  We can get one observation per row by metling the dataset.

In [28]:
molten = pd.melt(messy, 
                 id_vars=['year','artist','track','time','date entered'],
                 var_name = 'week',
                 value_name = 'rank',
                )
molten.sort_values(by=['date entered','week'], inplace=True)
molten.head()

year   artist           track  time date entered week  rank
5   2000  Aaliyah   I Don't Wanna  4:15   2000-01-29  wk1    84
13  2000  Aaliyah   I Don't Wanna  4:15   2000-01-29  wk2    62
21  2000  Aaliyah   I Don't Wanna  4:15   2000-01-29  wk3    51
0   2000    2,Pac  Baby Don't Cry  4:22   2000-02-26  wk1    87
8   2000    2,Pac  Baby Don't Cry  4:22   2000-02-26  wk2    82

We can clean the dataset further, first by turning week into number

In [29]:
molten['week'] = molten['week'].apply(lambda s: int(s[2:]))
molten.head()

year   artist           track  time date entered  week  rank
5   2000  Aaliyah   I Don't Wanna  4:15   2000-01-29     1    84
13  2000  Aaliyah   I Don't Wanna  4:15   2000-01-29     2    62
21  2000  Aaliyah   I Don't Wanna  4:15   2000-01-29     3    51
0   2000    2,Pac  Baby Don't Cry  4:22   2000-02-26     1    87
8   2000    2,Pac  Baby Don't Cry  4:22   2000-02-26     2    82

Second, we need the starting date of the week for each observation, instead of the date the track entered.

In [30]:
from datetime import datetime, timedelta

def increment_date(row):
    date = datetime.strptime(row['date entered'], "%Y-%m-%d")
    return date + timedelta(7) * (row['week'] - 1)

molten['date'] = molten.apply(increment_date, axis=1)
molten.drop('date entered', axis=1, inplace=True)
molten.head()

year   artist           track  time  week  rank       date
5   2000  Aaliyah   I Don't Wanna  4:15     1    84 2000-01-29
13  2000  Aaliyah   I Don't Wanna  4:15     2    62 2000-02-05
21  2000  Aaliyah   I Don't Wanna  4:15     3    51 2000-02-12
0   2000    2,Pac  Baby Don't Cry  4:22     1    87 2000-02-26
8   2000    2,Pac  Baby Don't Cry  4:22     2    82 2000-03-04

Last, this dataset is denormalized.  This is fine for most statistical and machine learning packages, but we might want to normalize it.  It means that we should group information that is repeated every week for a track in a separate table.  This information appears in columns `year ,artist, track, time`.  

In [31]:
tidy_track = molten[['year','artist','track','time']]\
            .groupby(['year','artist','track'])\
            .first()
tidy_track.reset_index(inplace=True)
tidy_track.reset_index(inplace=True)
tidy_track.rename(columns = {'index':'id'}, inplace=True)
tidy_track

id  year         artist                    track  time
0   0  2000          2,Pac           Baby Don't Cry  4:22
1   1  2000        2Ge+her  The Hardest Part Of ...  3:15
2   2  2000   3 Doors Down               Kryptonite  3:53
3   3  2000           98^0  Give Me Just One Nig...  3:24
4   4  2000        A*Teens            Dancing Queen  3:44
5   5  2000        Aaliyah            I Don't Wanna  4:15
6   6  2000        Aaliyah                Try Again  4:03
7   7  2000  Adams,Yolanda            Open My Heart  5:30

In [32]:
tidy_rank = pd.merge(molten, tidy_track, on='track')
tidy_rank = tidy_rank[['id', 'date', 'rank']]
tidy_rank.head()

id       date  rank
0   5 2000-01-29    84
1   5 2000-02-05    62
2   5 2000-02-12    51
3   0 2000-02-26    87
4   0 2000-03-04    82